# Partie 1.3 - Agregations avancees avec Spark SQL

Ce notebook realise des agregations avancees sur les donnees de consommation energetique
des batiments publics, en utilisant PySpark et Spark SQL.

**Objectifs :**
- Joindre les consommations avec le referentiel batiments
- Calculer l'intensite energetique (kWh/m2)
- Detecter les batiments hors norme
- Produire des totaux par commune et type de batiment
- Creer des vues SQL exploitables
- Sauvegarder la table agregee en Parquet

In [3]:
# --- Imports et creation de la session Spark ---

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import (
    StructType, StructField, StringType, DoubleType, IntegerType, TimestampType
)

# Creation de la session Spark avec configuration adaptee
spark = (
    SparkSession.builder
    .appName("ECF_Energie_Agregations")
    .master("local[*]")
    .config("spark.sql.shuffle.partitions", "8")
    .config("spark.driver.memory", "2g")
    .getOrCreate()
)

# Reduction du niveau de log pour la lisibilite
spark.sparkContext.setLogLevel("WARN")

print(f"Spark version : {spark.version}")
print("Session Spark initialisee avec succes.")

Spark version : 4.1.1
Session Spark initialisee avec succes.


## 1. Chargement des donnees nettoyees et du referentiel

In [4]:
# --- Chargement des consommations nettoyees (Parquet partitionne) ---

df_consommations = spark.read.parquet("../output/consommations_clean/")

print("=== Schema des consommations nettoyees ===")
df_consommations.printSchema()
print(f"Nombre d'enregistrements : {df_consommations.count():,}")
df_consommations.show(5, truncate=False)

# --- Chargement du referentiel batiments ---

df_batiments = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("../data/batiments.csv")
)

print("\n=== Schema du referentiel batiments ===")
df_batiments.printSchema()
print(f"Nombre de batiments : {df_batiments.count()}")
df_batiments.show(5, truncate=False)

=== Schema des consommations nettoyees ===
root
 |-- batiment_id: string (nullable = true)
 |-- unite: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- consommation: double (nullable = true)
 |-- annee: integer (nullable = true)
 |-- mois: integer (nullable = true)
 |-- jour: integer (nullable = true)
 |-- heure: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- annee_mois: string (nullable = true)
 |-- type_energie: string (nullable = true)

Nombre d'enregistrements : 7,491,733
+-----------+-----+-------------------+------------+-----+----+----+-----+----------+----------+------------+
|batiment_id|unite|timestamp          |consommation|annee|mois|jour|heure|date      |annee_mois|type_energie|
+-----------+-----+-------------------+------------+-----+----+----+-----+----------+----------+------------+
|BAT0001    |kWh  |2024-07-01 04:00:00|10.63       |2024 |7   |1   |4    |2024-07-01|2024-07   |electricite |
|BAT0001    |kWh  |2024-07-01 13:00:

## 2. Jointure consommations-batiments

In [5]:
# --- Jointure entre les consommations et le referentiel batiments ---
# On utilise une jointure interne (inner join) sur la cle batiment_id

df_joined = df_consommations.join(
    df_batiments,
    on="batiment_id",
    how="inner"
)

print("=== Schema apres jointure ===")
df_joined.printSchema()

print(f"Nombre d'enregistrements apres jointure : {df_joined.count():,}")

# Verification : aucune ligne perdue (tous les batiment_id existent dans le referentiel)
nb_conso = df_consommations.count()
nb_joined = df_joined.count()
print(f"Taux de correspondance : {nb_joined / nb_conso * 100:.1f}%")

# Apercu du resultat
df_joined.select(
    "batiment_id", "nom", "type", "commune", "surface_m2",
    "classe_energetique", "timestamp", "type_energie", "consommation", "unite"
).show(10, truncate=False)

# Mise en cache pour les traitements suivants
df_joined.cache()
print("DataFrame joint mis en cache.")

=== Schema apres jointure ===
root
 |-- batiment_id: string (nullable = true)
 |-- unite: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- consommation: double (nullable = true)
 |-- annee: integer (nullable = true)
 |-- mois: integer (nullable = true)
 |-- jour: integer (nullable = true)
 |-- heure: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- annee_mois: string (nullable = true)
 |-- type_energie: string (nullable = true)
 |-- nom: string (nullable = true)
 |-- type: string (nullable = true)
 |-- commune: string (nullable = true)
 |-- surface_m2: integer (nullable = true)
 |-- annee_construction: integer (nullable = true)
 |-- classe_energetique: string (nullable = true)
 |-- nb_occupants_moyen: integer (nullable = true)



Nombre d'enregistrements apres jointure : 7,491,733
Taux de correspondance : 100.0%
+-----------+-------------+-----+-------+----------+------------------+-------------------+------------+------------+-----+
|batiment_id|nom          |type |commune|surface_m2|classe_energetique|timestamp          |type_energie|consommation|unite|
+-----------+-------------+-----+-------+----------+------------------+-------------------+------------+------------+-----+
|BAT0001    |Ecole Paris 1|ecole|Paris  |1926      |E                 |2024-07-01 04:00:00|electricite |10.63       |kWh  |
|BAT0001    |Ecole Paris 1|ecole|Paris  |1926      |E                 |2024-07-01 13:00:00|electricite |160.53      |kWh  |
|BAT0001    |Ecole Paris 1|ecole|Paris  |1926      |E                 |2024-07-02 15:00:00|electricite |170.3       |kWh  |
|BAT0001    |Ecole Paris 1|ecole|Paris  |1926      |E                 |2024-07-02 19:00:00|electricite |135.46      |kWh  |
|BAT0001    |Ecole Paris 1|ecole|Paris  |1926   

## 3. Calcul de l'intensite energetique (kWh/m2)

In [6]:
# --- Calcul de l'intensite energetique ---
# L'intensite energetique rapporte la consommation a la surface du batiment.
# On filtre sur les consommations en kWh (electricite et gaz) pour avoir une unite homogene.

df_intensite = (
    df_joined
    .filter(F.col("unite") == "kWh")
    .withColumn(
        "intensite_kwh_m2",
        F.round(F.col("consommation") / F.col("surface_m2"), 4)
    )
)

# Agregation par batiment et type d'energie
df_intensite_agg = (
    df_intensite
    .groupBy("batiment_id", "nom", "type", "commune", "surface_m2",
             "classe_energetique", "type_energie")
    .agg(
        F.round(F.sum("consommation"), 2).alias("consommation_totale_kwh"),
        F.round(F.sum("intensite_kwh_m2"), 4).alias("intensite_totale_kwh_m2"),
        F.round(F.avg("intensite_kwh_m2"), 4).alias("intensite_moyenne_kwh_m2"),
        F.count("*").alias("nb_releves")
    )
    .orderBy(F.desc("intensite_totale_kwh_m2"))
)

print("=== Top 15 des batiments les plus consommateurs par m2 ===")
df_intensite_agg.show(15, truncate=False)

# Statistiques descriptives de l'intensite energetique
print("\n=== Statistiques de l'intensite energetique (kWh/m2) par type d'energie ===")
(
    df_intensite_agg
    .groupBy("type_energie")
    .agg(
        F.round(F.avg("intensite_moyenne_kwh_m2"), 4).alias("moyenne"),
        F.round(F.stddev("intensite_moyenne_kwh_m2"), 4).alias("ecart_type"),
        F.round(F.min("intensite_moyenne_kwh_m2"), 4).alias("min"),
        F.round(F.max("intensite_moyenne_kwh_m2"), 4).alias("max")
    )
    .show(truncate=False)
)

=== Top 15 des batiments les plus consommateurs par m2 ===


+-----------+-------------------------+-------+-------------+----------+------------------+------------+-----------------------+-----------------------+------------------------+----------+
|batiment_id|nom                      |type   |commune      |surface_m2|classe_energetique|type_energie|consommation_totale_kwh|intensite_totale_kwh_m2|intensite_moyenne_kwh_m2|nb_releves|
+-----------+-------------------------+-------+-------------+----------+------------------+------------+-----------------------+-----------------------+------------------------+----------+
|BAT0122    |Piscine Le Havre 122     |piscine|Le Havre     |1579      |G                 |gaz         |1.480386422E7          |9375.4665              |0.5475                  |17125     |
|BAT0134    |Piscine Saint-Etienne 134|piscine|Saint-Etienne|2126      |G                 |gaz         |1.99127683E7           |9366.2998              |0.5464                  |17142     |
|BAT0121    |Piscine Le Havre 121     |piscine|Le Havre

+------------+-------+----------+------+------+
|type_energie|moyenne|ecart_type|min   |max   |
+------------+-------+----------+------+------+
|gaz         |0.1919 |0.1297    |0.0317|0.548 |
|electricite |0.1288 |0.0862    |0.0211|0.3651|
+------------+-------+----------+------+------+



## 4. Detection des batiments hors norme

In [7]:
# --- Detection des batiments hors norme ---
# Un batiment est considere "hors norme" si sa consommation totale depasse
# 3 fois la mediane de sa categorie (type de batiment x type d'energie).

# Etape 1 : Agreger la consommation totale par batiment et type d'energie
df_conso_totale = (
    df_joined
    .filter(F.col("unite") == "kWh")
    .groupBy("batiment_id", "nom", "type", "commune", "surface_m2",
             "classe_energetique", "type_energie")
    .agg(
        F.round(F.sum("consommation"), 2).alias("consommation_totale")
    )
)

# Etape 2 : Calculer la mediane par categorie (type de batiment x type d'energie)
# On utilise percentile_approx pour calculer la mediane
df_medianes = (
    df_conso_totale
    .groupBy("type", "type_energie")
    .agg(
        F.expr("percentile_approx(consommation_totale, 0.5)").alias("mediane_consommation"),
        F.round(F.avg("consommation_totale"), 2).alias("moyenne_consommation"),
        F.count("*").alias("nb_batiments_categorie")
    )
)

print("=== Medianes de consommation par categorie ===")
df_medianes.orderBy("type", "type_energie").show(truncate=False)

# Etape 3 : Joindre et identifier les batiments hors norme (> 3x la mediane)
df_hors_norme = (
    df_conso_totale
    .join(df_medianes, on=["type", "type_energie"], how="inner")
    .withColumn(
        "seuil_hors_norme",
        F.round(F.col("mediane_consommation") * 3, 2)
    )
    .withColumn(
        "ratio_vs_mediane",
        F.round(F.col("consommation_totale") / F.col("mediane_consommation"), 2)
    )
    .filter(F.col("consommation_totale") > F.col("seuil_hors_norme"))
    .orderBy(F.desc("ratio_vs_mediane"))
)

nb_hors_norme = df_hors_norme.count()
print(f"\n=== Batiments hors norme detectes : {nb_hors_norme} ===")
df_hors_norme.select(
    "batiment_id", "nom", "type", "commune", "type_energie",
    "consommation_totale", "mediane_consommation", "seuil_hors_norme",
    "ratio_vs_mediane", "classe_energetique", "surface_m2"
).show(50, truncate=False)

if nb_hors_norme == 0:
    print("Aucun batiment hors norme detecte avec le seuil de 3x la mediane.")

=== Medianes de consommation par categorie ===


+-----------+------------+--------------------+--------------------+----------------------+
|type       |type_energie|mediane_consommation|moyenne_consommation|nb_batiments_categorie|
+-----------+------------+--------------------+--------------------+----------------------+
|ecole      |electricite |2170449.85          |2338034.57          |27                    |
|ecole      |gaz         |3268341.61          |3510450.46          |27                    |
|gymnase    |electricite |4311552.65          |4340835.57          |27                    |
|gymnase    |gaz         |6477337.69          |6516863.36          |27                    |
|mairie     |electricite |1035307.45          |1046960.72          |31                    |
|mairie     |gaz         |1553589.25          |1570372.62          |31                    |
|mediatheque|electricite |1672339.04          |1899966.47          |30                    |
|mediatheque|gaz         |2392333.82          |2713558.96          |30          


=== Batiments hors norme detectes : 0 ===


+-----------+---+----+-------+------------+-------------------+--------------------+----------------+----------------+------------------+----------+
|batiment_id|nom|type|commune|type_energie|consommation_totale|mediane_consommation|seuil_hors_norme|ratio_vs_mediane|classe_energetique|surface_m2|
+-----------+---+----+-------+------------+-------------------+--------------------+----------------+----------------+------------------+----------+
+-----------+---+----+-------+------------+-------------------+--------------------+----------------+----------------+------------------+----------+

Aucun batiment hors norme detecte avec le seuil de 3x la mediane.


## 5. Totaux par commune et type de batiment

In [8]:
# --- Totaux par commune et type de batiment ---
# Agregation croisee : consommation totale et moyenne par commune et type de batiment

df_totaux_commune = (
    df_joined
    .filter(F.col("unite") == "kWh")
    .groupBy("commune", "type")
    .agg(
        F.round(F.sum("consommation"), 2).alias("consommation_totale_kwh"),
        F.round(F.avg("consommation"), 2).alias("consommation_moyenne_kwh"),
        F.countDistinct("batiment_id").alias("nb_batiments"),
        F.count("*").alias("nb_releves")
    )
    .orderBy("commune", "type")
)

print("=== Totaux par commune et type de batiment ===")
df_totaux_commune.show(50, truncate=False)

# --- Tableau croise (pivot) : consommation totale par commune x type de batiment ---
print("\n=== Tableau croise : consommation totale (kWh) par commune et type de batiment ===")

# Recuperer les types de batiments distincts pour le pivot
types_batiments = (
    df_joined.select("type").distinct()
    .rdd.flatMap(lambda x: x).collect()
)
types_batiments.sort()

df_pivot = (
    df_joined
    .filter(F.col("unite") == "kWh")
    .groupBy("commune")
    .pivot("type", types_batiments)
    .agg(F.round(F.sum("consommation"), 2))
    .orderBy("commune")
    .na.fill(0)
)

df_pivot.show(truncate=False)

# --- Tableau croise : consommation moyenne par commune x type de batiment ---
print("\n=== Tableau croise : consommation moyenne (kWh) par commune et type de batiment ===")

df_pivot_avg = (
    df_joined
    .filter(F.col("unite") == "kWh")
    .groupBy("commune")
    .pivot("type", types_batiments)
    .agg(F.round(F.avg("consommation"), 2))
    .orderBy("commune")
    .na.fill(0)
)

df_pivot_avg.show(truncate=False)

=== Totaux par commune et type de batiment ===


+-----------+-----------+-----------------------+------------------------+------------+----------+
|commune    |type       |consommation_totale_kwh|consommation_moyenne_kwh|nb_batiments|nb_releves|
+-----------+-----------+-----------------------+------------------------+------------+----------+
|Bordeaux   |ecole      |1.118413525E7          |163.37                  |2           |68458     |
|Bordeaux   |gymnase    |1.236730429E7          |361.93                  |1           |34170     |
|Bordeaux   |mairie     |6952408.3              |67.73                   |3           |102652    |
|Bordeaux   |mediatheque|8896398.13             |260.14                  |1           |34198     |
|Bordeaux   |piscine    |3.552010911E7          |1039.33                 |1           |34176     |
|Le Havre   |ecole      |1.258754701E7          |184.1                   |2           |68374     |
|Le Havre   |gymnase    |1.134556331E7          |332.04                  |1           |34169     |
|Le Havre 

+-------------+-------------+-------------+-------------+-------------+--------------+
|commune      |ecole        |gymnase      |mairie       |mediatheque  |piscine       |
+-------------+-------------+-------------+-------------+-------------+--------------+
|Bordeaux     |1.118413525E7|1.236730429E7|6952408.3    |8896398.13   |3.552010911E7 |
|Le Havre     |1.258754701E7|1.134556331E7|2386547.64   |3662253.31   |1.4863671926E8|
|Lille        |0.0          |9608495.69   |1947175.54   |6056186.63   |1.1902687005E8|
|Lyon         |2.175438804E7|4.227186305E7|1.105494578E7|6584926.67   |4.127922187E7 |
|Marseille    |2.244732097E7|4774154.6    |6765970.32   |8724761.27   |5.939170593E7 |
|Montpellier  |3977709.4    |2.51714189E7 |5045840.37   |8845871.71   |3.892680763E7 |
|Nantes       |1.820928396E7|5524912.57   |1.439925786E7|8470817.46   |1.59007049E7  |
|Nice         |1.810784455E7|3.982834355E7|4633953.92   |1.230211974E7|5.968303375E7 |
|Paris        |1.258463573E7|2.16842793E7 |

+-------------+------+-------+------+-----------+-------+
|commune      |ecole |gymnase|mairie|mediatheque|piscine|
+-------------+------+-------+------+-----------+-------+
|Bordeaux     |163.37|361.93 |67.73 |260.14     |1039.33|
|Le Havre     |184.1 |332.04 |34.87 |107.04     |868.89 |
|Lille        |0.0   |280.56 |56.79 |59.04      |1159.36|
|Lyon         |159.04|411.46 |107.67|192.26     |1208.3 |
|Marseille    |164.06|139.32 |98.94 |127.54     |868.8  |
|Montpellier  |116.39|245.25 |73.79 |129.22     |1137.88|
|Nantes       |133.14|161.49 |105.31|123.84     |464.02 |
|Nice         |264.21|388.14 |67.7  |179.76     |581.02 |
|Paris        |122.69|317.02 |79.92 |63.2       |1783.43|
|Reims        |235.69|316.16 |50.61 |180.53     |1411.52|
|Rennes       |0.0   |0.0    |68.16 |153.74     |714.11 |
|Saint-Etienne|0.0   |292.81 |31.2  |40.8       |1163.39|
|Strasbourg   |191.42|359.03 |101.52|0.0        |1123.94|
|Toulon       |273.42|329.37 |0.0   |119.27     |980.12 |
|Toulouse     

## 6. Creation de vues SQL exploitables

In [9]:
# --- Enregistrement des DataFrames comme vues temporaires Spark SQL ---

df_joined.createOrReplaceTempView("consommations_batiments")
df_batiments.createOrReplaceTempView("batiments")
df_intensite_agg.createOrReplaceTempView("intensite_energetique")

print("Vues temporaires creees : consommations_batiments, batiments, intensite_energetique")
print("="*80)

# --- Requete 1 : Top 10 des batiments par consommation totale ---
print("\n=== REQUETE 1 : Top 10 des batiments par consommation totale (kWh) ===")

df_top10 = spark.sql("""
    SELECT
        batiment_id,
        nom,
        type,
        commune,
        classe_energetique,
        surface_m2,
        ROUND(SUM(consommation), 2) AS consommation_totale_kwh
    FROM consommations_batiments
    WHERE unite = 'kWh'
    GROUP BY batiment_id, nom, type, commune, classe_energetique, surface_m2
    ORDER BY consommation_totale_kwh DESC
    LIMIT 10
""")

df_top10.show(truncate=False)

# --- Requete 2 : Consommation moyenne par classe DPE ---
print("\n=== REQUETE 2 : Consommation moyenne par classe energetique (DPE) ===")

df_dpe = spark.sql("""
    SELECT
        classe_energetique,
        COUNT(DISTINCT batiment_id) AS nb_batiments,
        ROUND(AVG(consommation), 2) AS consommation_moyenne_kwh,
        ROUND(SUM(consommation), 2) AS consommation_totale_kwh,
        ROUND(AVG(consommation / surface_m2), 4) AS intensite_moyenne_kwh_m2
    FROM consommations_batiments
    WHERE unite = 'kWh'
    GROUP BY classe_energetique
    ORDER BY classe_energetique
""")

df_dpe.show(truncate=False)

# --- Requete 3 : Evolution mensuelle par commune ---
print("\n=== REQUETE 3 : Evolution mensuelle de la consommation par commune ===")

df_mensuel = spark.sql("""
    SELECT
        commune,
        DATE_FORMAT(timestamp, 'yyyy-MM') AS mois,
        ROUND(SUM(consommation), 2) AS consommation_totale_kwh,
        COUNT(DISTINCT batiment_id) AS nb_batiments_actifs,
        COUNT(*) AS nb_releves
    FROM consommations_batiments
    WHERE unite = 'kWh'
    GROUP BY commune, DATE_FORMAT(timestamp, 'yyyy-MM')
    ORDER BY commune, mois
""")

df_mensuel.show(30, truncate=False)

# --- Requete 4 : Comparaison electricite vs gaz par type de batiment ---
print("\n=== REQUETE 4 : Comparaison electricite vs gaz par type de batiment ===")

df_elec_gaz = spark.sql("""
    SELECT
        type,
        type_energie,
        COUNT(DISTINCT batiment_id) AS nb_batiments,
        ROUND(SUM(consommation), 2) AS consommation_totale_kwh,
        ROUND(AVG(consommation), 2) AS consommation_moyenne_kwh,
        ROUND(SUM(consommation) / COUNT(DISTINCT batiment_id), 2) AS conso_totale_par_batiment
    FROM consommations_batiments
    WHERE unite = 'kWh'
      AND type_energie IN ('electricite', 'gaz')
    GROUP BY type, type_energie
    ORDER BY type, type_energie
""")

df_elec_gaz.show(truncate=False)

# Ratio electricite/gaz par type de batiment
print("\n=== Ratio electricite/gaz par type de batiment ===")

df_ratio = spark.sql("""
    WITH conso_par_energie AS (
        SELECT
            type,
            type_energie,
            SUM(consommation) AS consommation_totale
        FROM consommations_batiments
        WHERE unite = 'kWh'
          AND type_energie IN ('electricite', 'gaz')
        GROUP BY type, type_energie
    )
    SELECT
        e.type,
        ROUND(e.consommation_totale, 2) AS total_electricite,
        ROUND(g.consommation_totale, 2) AS total_gaz,
        ROUND(e.consommation_totale / g.consommation_totale, 2) AS ratio_elec_gaz
    FROM conso_par_energie e
    JOIN conso_par_energie g
        ON e.type = g.type
    WHERE e.type_energie = 'electricite'
      AND g.type_energie = 'gaz'
    ORDER BY ratio_elec_gaz DESC
""")

df_ratio.show(truncate=False)

Vues temporaires creees : consommations_batiments, batiments, intensite_energetique

=== REQUETE 1 : Top 10 des batiments par consommation totale (kWh) ===


+-----------+-------------------------+-------+-------------+------------------+----------+-----------------------+
|batiment_id|nom                      |type   |commune      |classe_energetique|surface_m2|consommation_totale_kwh|
+-----------+-------------------------+-------+-------------+------------------+----------+-----------------------+
|BAT0005    |Piscine Paris 5          |piscine|Paris        |G                 |3913      |6.100768928E7          |
|BAT0048    |Piscine Lille 48         |piscine|Lille        |G                 |3754      |5.835917104E7          |
|BAT0136    |Piscine Toulon 136       |piscine|Toulon       |F                 |3926      |5.203454169E7          |
|BAT0112    |Piscine Reims 112        |piscine|Reims        |G                 |3095      |4.820487832E7          |
|BAT0133    |Piscine Saint-Etienne 133|piscine|Saint-Etienne|F                 |3507      |4.652460348E7          |
|BAT0050    |Piscine Lille 50         |piscine|Lille        |F          

+------------------+------------+------------------------+-----------------------+------------------------+
|classe_energetique|nb_batiments|consommation_moyenne_kwh|consommation_totale_kwh|intensite_moyenne_kwh_m2|
+------------------+------------+------------------------+-----------------------+------------------------+
|A                 |2           |54.13                   |3703991.8              |0.0326                  |
|B                 |5           |238.01                  |4.070759887E7          |0.0944                  |
|C                 |15          |151.19                  |7.758545992E7          |0.0876                  |
|D                 |23          |288.51                  |2.2709009392E8         |0.1284                  |
|E                 |24          |250.5                   |2.0561920529E8         |0.1306                  |
|F                 |37          |441.51                  |5.5893018446E8         |0.1964                  |
|G                 |40      

+--------+-------+-----------------------+-------------------+----------+
|commune |mois   |consommation_totale_kwh|nb_batiments_actifs|nb_releves|
+--------+-------+-----------------------+-------------------+----------+
|Bordeaux|2023-01|4530660.46             |8                  |11615     |
|Bordeaux|2023-02|4065536.08             |8                  |10457     |
|Bordeaux|2023-03|2853925.93             |8                  |11569     |
|Bordeaux|2023-04|2704851.61             |8                  |11226     |
|Bordeaux|2023-05|2873154.82             |8                  |11650     |
|Bordeaux|2023-06|1930885.27             |8                  |11226     |
|Bordeaux|2023-07|1964098.51             |8                  |11625     |
|Bordeaux|2023-08|2015083.87             |8                  |11664     |
|Bordeaux|2023-09|2718694.78             |8                  |11249     |
|Bordeaux|2023-10|2820013.88             |8                  |11587     |
|Bordeaux|2023-11|4397491.16          

+-----------+-----------------+--------------+--------------+
|type       |total_electricite|total_gaz     |ratio_elec_gaz|
+-----------+-----------------+--------------+--------------+
|mediatheque|5.699899395E7    |8.140676874E7 |0.7           |
|piscine    |4.068716451E8    |6.1021197186E8|0.67          |
|ecole      |6.312693335E7    |9.478216242E7 |0.67          |
|gymnase    |1.1720256029E8   |1.7595531079E8|0.67          |
|mairie     |3.245578239E7    |4.868155131E7 |0.67          |
+-----------+-----------------+--------------+--------------+



## 7. Sauvegarde de la table agregee

In [10]:
# --- Construction de la table agregee finale ---
# On regroupe toutes les informations utiles pour les analyses en aval

df_agregee = (
    df_joined
    .filter(F.col("unite") == "kWh")
    .withColumn("mois", F.date_format(F.col("timestamp"), "yyyy-MM"))
    .groupBy(
        "batiment_id", "nom", "type", "commune", "surface_m2",
        "annee_construction", "classe_energetique", "nb_occupants_moyen",
        "type_energie", "mois"
    )
    .agg(
        F.round(F.sum("consommation"), 2).alias("consommation_totale_kwh"),
        F.round(F.avg("consommation"), 2).alias("consommation_moyenne_kwh"),
        F.round(F.min("consommation"), 2).alias("consommation_min_kwh"),
        F.round(F.max("consommation"), 2).alias("consommation_max_kwh"),
        F.count("*").alias("nb_releves"),
        F.round(F.sum("consommation") / F.first("surface_m2"), 4).alias("intensite_kwh_m2")
    )
    .orderBy("commune", "type", "batiment_id", "type_energie", "mois")
)

print(f"=== Table agregee : {df_agregee.count():,} lignes ===")
df_agregee.printSchema()
df_agregee.show(10, truncate=False)

# --- Sauvegarde en Parquet ---
chemin_sortie = "../output/consommations_agregees.parquet"

(
    df_agregee
    .write
    .mode("overwrite")
    .parquet(chemin_sortie)
)

print(f"\nTable agregee sauvegardee avec succes : {chemin_sortie}")
print(f"Nombre de colonnes : {len(df_agregee.columns)}")
print(f"Colonnes : {df_agregee.columns}")

=== Table agregee : 7,008 lignes ===
root
 |-- batiment_id: string (nullable = true)
 |-- nom: string (nullable = true)
 |-- type: string (nullable = true)
 |-- commune: string (nullable = true)
 |-- surface_m2: integer (nullable = true)
 |-- annee_construction: integer (nullable = true)
 |-- classe_energetique: string (nullable = true)
 |-- nb_occupants_moyen: integer (nullable = true)
 |-- type_energie: string (nullable = true)
 |-- mois: string (nullable = true)
 |-- consommation_totale_kwh: double (nullable = true)
 |-- consommation_moyenne_kwh: double (nullable = true)
 |-- consommation_min_kwh: double (nullable = true)
 |-- consommation_max_kwh: double (nullable = true)
 |-- nb_releves: long (nullable = false)
 |-- intensite_kwh_m2: double (nullable = true)



+-----------+-----------------+-----+--------+----------+------------------+------------------+------------------+------------+-------+-----------------------+------------------------+--------------------+--------------------+----------+----------------+
|batiment_id|nom              |type |commune |surface_m2|annee_construction|classe_energetique|nb_occupants_moyen|type_energie|mois   |consommation_totale_kwh|consommation_moyenne_kwh|consommation_min_kwh|consommation_max_kwh|nb_releves|intensite_kwh_m2|
+-----------+-----------------+-----+--------+----------+------------------+------------------+------------------+------------+-------+-----------------------+------------------------+--------------------+--------------------+----------+----------------+
|BAT0045    |Ecole Bordeaux 45|ecole|Bordeaux|1203      |2001              |E                 |131               |electricite |2023-01|91467.75               |125.3                   |2.84                |336.9               |730      


Table agregee sauvegardee avec succes : ../output/consommations_agregees.parquet
Nombre de colonnes : 16
Colonnes : ['batiment_id', 'nom', 'type', 'commune', 'surface_m2', 'annee_construction', 'classe_energetique', 'nb_occupants_moyen', 'type_energie', 'mois', 'consommation_totale_kwh', 'consommation_moyenne_kwh', 'consommation_min_kwh', 'consommation_max_kwh', 'nb_releves', 'intensite_kwh_m2']


## Conclusion

Ce notebook a permis de realiser les traitements suivants :

1. **Jointure** des consommations nettoyees avec le referentiel batiments (146 batiments)
2. **Intensite energetique** : calcul du ratio kWh/m2 pour identifier les batiments les plus consommateurs par unite de surface
3. **Detection des anomalies** : identification des batiments dont la consommation depasse 3 fois la mediane de leur categorie
4. **Agregations croisees** : totaux et moyennes par commune et type de batiment, avec tableaux croises
5. **Vues SQL** : requetes analytiques exploitables (top consommateurs, analyse DPE, evolution mensuelle, comparaison electricite/gaz)
6. **Sauvegarde** : table agregee en format Parquet pour les etapes suivantes du pipeline

In [11]:
# --- Arret de la session Spark ---
spark.stop()
print("Session Spark arretee.")

Session Spark arretee.
